### 4.2. XGBoost 预测

predict 函数：取 2020-01-11 到 last_date 之间的各种数据，使用前 n - 1 天的数据训练并预测各地最后一天的新增确诊人数。

In [1]:
from coronavirus_analyzer import CoronavirusAnalyzer
import numpy as np
import pandas as pd
import xgboost as xgb

def predict(last_date, use_move=True, use_ma3=False, use_in_out_rate=True, use_day_idx=True, 
            use_move_in_inc_rate=False, use_weather=False, omit_hubei=True, selected_regions=None):
    '''
    last_date               加载数据的最后一天
    use_move                是否使用人流数据，函数中 use_data 是算法使用的数据
    use_ma3                 是否使用前3天（不含当天）新增确诊人数均值，函数中 use_data 是算法使用的数据
    use_in_out_rate         各地 1天前的人流进出比例
    use_weather             是否使用天气数据，函数中 use_data 是算法使用的数据
    omit_hubei              是否忽略湖北
    selected_regions        只学习和预测某些地区，如果为 None，表示都学习和预测
    
    use_move_in_inc_rate 同时启用，会有河南、江西 2020-01-01 预测的严重错误
    '''
    omitted_regions = ['湖北'] if omit_hubei else []
    
    analyzer = CoronavirusAnalyzer(last_date)
    df_virus_daily_inc_injured = analyzer.del_city_special_regions(analyzer.df_virus_daily_inc_injured)
    df_move_in_injured = analyzer.del_city_regions(analyzer.df_move_in_injured)
    df_move_in_injured_2 = analyzer.del_city_regions(analyzer.get_df_move_in_injured(2))
    df_move_in_injured_3 = analyzer.del_city_regions(analyzer.get_df_move_in_injured(3))
    df_move_in_injured_4 = analyzer.del_city_regions(analyzer.get_df_move_in_injured(4))
    df_move_in_injured_5 = analyzer.del_city_regions(analyzer.get_df_move_in_injured(5))
    df_weather_ma = analyzer.del_city_special_regions(analyzer.df_weather_ma)
#     df_virus_daily_injured = analyzer.del_city_special_regions(analyzer.df_virus_daily_injured)
    df_virus_7_days = analyzer.del_city_special_regions(analyzer.df_virus_7_days_inc_injured)
    df_daily_inc_ma3 = analyzer.moving_avg(analyzer.df_virus_daily_inc_injured, window=3, 
                                           shift=1, keep_shape=True).fillna(0)
    df_curve_in_out_rate = analyzer.df_curve_in_out_rate
    df_move_in_injured_inc_rate_1 = analyzer.get_df_move_in_injured_inc_rate(1)
    df_move_in_injured_inc_rate_2 = analyzer.get_df_move_in_injured_inc_rate(2)
    df_move_in_injured_inc_rate_3 = analyzer.get_df_move_in_injured_inc_rate(3)
    df_move_in_injured_inc_rate_4 = analyzer.get_df_move_in_injured_inc_rate(4)
    df_move_in_injured_inc_rate_5 = analyzer.get_df_move_in_injured_inc_rate(5)
    
    # 自然数日期
    df_day_idx = analyzer.get_df_day_idx(df_move_in_injured)
    # 1 级列索引转 2 级列索引
#     if not isinstance(df_move_in_injured.columns, pd.MultiIndex):
    df_day_idx.columns = pd.MultiIndex.from_product(
        [df_day_idx.columns, ['天数']])
    df_move_in_injured.columns = pd.MultiIndex.from_product(
        [df_move_in_injured.columns, ['1天前人流风险系数']])
    df_move_in_injured_2.columns = pd.MultiIndex.from_product(
        [df_move_in_injured_2.columns, ['2天前人流风险系数']])
    df_move_in_injured_3.columns = pd.MultiIndex.from_product(
        [df_move_in_injured_3.columns, ['3天前人流风险系数']])
    df_move_in_injured_4.columns = pd.MultiIndex.from_product(
        [df_move_in_injured_4.columns, ['4天前人流风险系数']])
    df_move_in_injured_5.columns = pd.MultiIndex.from_product(
        [df_move_in_injured_5.columns, ['5天前人流风险系数']])
#     df_virus_daily_injured.columns = pd.MultiIndex.from_product(
#         [df_virus_daily_injured.columns, ['累计确诊']])
    df_move_in_injured_inc_rate_1.columns = pd.MultiIndex.from_product(
        [df_move_in_injured_inc_rate_1.columns, ['风险系数1日增长趋势']])
    df_move_in_injured_inc_rate_2.columns = pd.MultiIndex.from_product(
        [df_move_in_injured_inc_rate_2.columns, ['风险系数2日增长趋势']])
    df_move_in_injured_inc_rate_3.columns = pd.MultiIndex.from_product(
        [df_move_in_injured_inc_rate_3.columns, ['风险系数3日增长趋势']])
    df_move_in_injured_inc_rate_4.columns = pd.MultiIndex.from_product(
        [df_move_in_injured_inc_rate_4.columns, ['风险系数4日增长趋势']])
    df_move_in_injured_inc_rate_5.columns = pd.MultiIndex.from_product(
        [df_move_in_injured_inc_rate_5.columns, ['风险系数5日增长趋势']])
    df_daily_inc_ma3.columns = pd.MultiIndex.from_product(
        [df_daily_inc_ma3.columns, ['3日新增均值']])
    df_curve_in_out_rate.columns = pd.MultiIndex.from_product(
        [df_curve_in_out_rate.columns, ['进出比例']])
    # index 统一从 2020-01-11 到 last_date，并合并 4 个 DataFrame
    index = df_move_in_injured.index
    dfs = []
    
    # 所有使用的数据 list
    use_data = [df_virus_7_days]  # 自然人天数和各地 1天前、2天前、......、7天前的每日新增确诊人数，一定会使用
    if use_day_idx:
        # 自然数日期
        use_data.append(df_day_idx)
    if use_weather:
        # 各地 14天前、13天前、......、3天前的天气滑动加权平均数据，权重值为 1,2,3,4,5,6,7,8,9,10,9,8
        use_data.append(df_weather_ma)
    if use_move:
        use_data.append(df_move_in_injured)
        use_data.append(df_move_in_injured_2)
        use_data.append(df_move_in_injured_3)
        use_data.append(df_move_in_injured_4)
        use_data.append(df_move_in_injured_5)
        # 各地 7 天内（不含当天，即 1天前到7天前，的所有确诊人数作为权重 * 进入该地区的人流规模）
    if use_ma3:
        # 各地 1天前、2天前、3天前 每日新增均值
        use_data.append(df_daily_inc_ma3)
    if use_in_out_rate:
        # 各地 1天前的人流进出比例
        use_data.append(df_curve_in_out_rate)
    if use_move_in_inc_rate:
        # 各地风险系数增长趋势
        use_data.append(df_move_in_injured_inc_rate_1)
        use_data.append(df_move_in_injured_inc_rate_2)
        use_data.append(df_move_in_injured_inc_rate_3)
#         use_data.append(df_move_in_injured_inc_rate_5)
    for df in use_data:
        for region in omitted_regions:
            try:
                del df[region]
            except:
                pass
        if df.shape[0] != index.size:
            df = df.reindex(index)
        dfs.append(df)
    df_trait = pd.concat(dfs, axis=1, sort=False)
    
#     try:
#         print('============== {}'.format(df_trait.columns))
    df_trait = df_trait.sort_index(axis=1)
#     except:
#         print('============== except')

    df_X_train = df_trait.iloc[:-1]
#     df_y_train = df_virus_daily_inc_injured.iloc[:-1]
    df_y_train = df_virus_daily_inc_injured.iloc[:df_X_train.shape[0]]
    df_X_test = df_trait.iloc[-1:]
    df_y_test = df_virus_daily_inc_injured.iloc[-1:]
    X_train = None
    y_train = None
    X_test = None
    y_test = None
    regions = []
    for region in df_y_train.columns:
        if selected_regions is not None and region not in selected_regions:
            continue
        if region not in omitted_regions:
            regions.append(region)
            arr_X_train = df_X_train[region].values
            arr_y_train = df_y_train[region].values
            arr_X_test = df_X_test[region].values
            arr_y_test = df_y_test[region].values
            if X_train is None:
                X_train = arr_X_train
                y_train = arr_y_train
                X_test = arr_X_test
                y_test = arr_y_test
            else:
                X_train = np.vstack([X_train, arr_X_train])
                y_train = np.hstack([y_train, arr_y_train])
                X_test = np.vstack([X_test, arr_X_test])
                y_test = np.hstack([y_test, arr_y_test])
#     print(X_train.shape, y_train.shape)
    region_cnt = len(regions)

    min_err_rate = 1
    min_err_sum = 1000000
    best_objective = best_max_depth = best_n_estimators = best_learning_rate = None
    objectives = ['reg:gamma']  # 'reg:squarederror', 'count:poisson', 
    for objective in objectives:
#         print('*' * 40, objective, '*' * 40)
        for max_depth in range(10, 22, 2):  # (10, 16):
#             print('max_depth: {}'.format(max_depth))
            for n_estimators in range(200, 530, 30):  # (100, 210, 10):
        #         print('n_estimators: {}'.format(n_estimators))
                for learning_rate in range(10, 11):
                    learning_rate = learning_rate / 100
                    model = xgb.XGBRegressor(
                        max_depth=max_depth, 
                        learning_rate=learning_rate, 
                        n_estimators=n_estimators, 
                        objective=objective)
                    model.fit(X_train, y_train)
                    
                    df_predict = pd.DataFrame(np.array([model.predict(X_test), y_test]).T, 
                                              index=regions, columns=['预测', '实际'])
                    df_predict['预测差'] = df_predict['预测'] - df_predict['实际']
                    df_predict['预测误差百分比'] = df_predict['预测差'] / df_predict['实际']
                    df_predict['预测误差百分比'][df_predict['实际'].values == 0] = 0
                    df_predict['权重'] = region_cnt * df_predict['实际'] / df_predict['实际'].sum()
                    df_predict['加权预测误差百分比'] = df_predict['预测误差百分比'] * df_predict['权重']
                    err_rate = abs(df_predict['加权预测误差百分比'].values).sum() / region_cnt
                    err_sum = abs(df_predict['预测差'].values).sum()
                    if err_rate < min_err_rate or err_sum < min_err_sum:
    #                     print('max_depth：{}，n_estimators：{}，learning_rate：{}，加权预测误差绝对值的均值：{}，'\
    #                           '最小误差绝对值总和为：{}。'
    #                           .format(max_depth, n_estimators, learning_rate, err_rate, err_sum))
                        if min_err_rate > err_rate:
                            min_err_rate = err_rate
                            min_err_sum = err_sum
                            best_max_depth = max_depth
                            best_n_estimators = n_estimators
                            best_learning_rate = learning_rate
                            best_objective = objective
    print('best_objective：{}，best_max_depth：{}，best_n_estimators：{}，best_learning_rate：{}，'\
          '加权预测误差绝对值的均值：{}，最小误差绝对值总和为：{}。'
          .format(best_objective, best_max_depth, best_n_estimators, best_learning_rate, min_err_rate, min_err_sum))

#     print(best_max_depth, best_learning_rate, best_n_estimators, best_objective)
    model = xgb.XGBRegressor(
        max_depth=best_max_depth,
        learning_rate=best_learning_rate,
        n_estimators=best_n_estimators,
        objective=best_objective)
    model.fit(X_train, y_train)
    df_predict = pd.DataFrame(np.array([model.predict(X_test), y_test]).T, 
                              index=regions, columns=['预测', '实际'])
    
    df_predict['预测差'] = df_predict['预测'] - df_predict['实际']
    df_predict['预测误差百分比'] = df_predict['预测差'] / df_predict['实际']
    df_predict['预测误差百分比'][df_predict['实际'].values == 0] = 0
    df_predict['权重'] = region_cnt * df_predict['实际'] / df_predict['实际'].sum()
    df_predict['加权预测误差百分比'] = df_predict['预测误差百分比'] * df_predict['权重']
    err_rate = abs(df_predict['加权预测误差百分比'].values).sum() / region_cnt
    
    err_percent_sum = df_predict['预测误差百分比'].values.sum()
    err_sum = df_predict['预测差'].values.sum()
    err_abs_sum = abs(df_predict['预测差'].values).sum()
    print('预测：{}，预测误差百分比总和：{}，预测差总和：{}，预测差绝对值的总和：{}，加权预测误差绝对值的均值：{}'
          .format(last_date, err_percent_sum, err_sum, err_abs_sum, err_rate))
    df_predict = df_predict.sort_values('实际', ascending=False)
    return df_predict

In [2]:
# [1] 32 19 29 [2] 32 18 28 [3] 27 18 30 [1&3] 26 22 28\22 22 28
# 38 21 27 | 37 21 27 | 25 19 29 | 33 26 26 | 22 17 31 | 19 20 30 | 17 19 31 | 22 20 31
import datetime
date = datetime.date(2020, 2, 3)
dfs_predict = {}
while date <= datetime.date.today():
    str_date = str(date)
    dfs_predict[str_date] = predict(str_date)
    date += datetime.timedelta(days=1)

2020-02-04 23:42:56,405 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.
2020-02-04 23:42:56,650 - CoronavirusAnalyzer - WARNING - 在最后一天（2020-02-03），如下这些地区没有新增的确诊人数：西藏，如下这些地区没有任何疫情数据变化：西藏。请确保这些地区已经公布了最后一天的数据（一般是后面一天上午公布），否则分析出来的结果可能不准确。


best_objective：reg:gamma，best_max_depth：14，best_n_estimators：350，best_learning_rate：0.1，加权预测误差绝对值的均值：0.3103341508372787，最小误差绝对值总和为：272.783718585968。


2020-02-04 23:43:48,218 - CoronavirusAnalyzer - WARNING - 在最后一天（2020-02-04），如下这些地区没有新增的确诊人数：西藏，如下这些地区没有任何疫情数据变化：西藏。请确保这些地区已经公布了最后一天的数据（一般是后面一天上午公布），否则分析出来的结果可能不准确。


预测：2020-02-03，预测误差百分比总和：9.581998736992606，预测差总和：-119.4936671257019，预测差绝对值的总和：272.783718585968，加权预测误差绝对值的均值：0.3103341508372787
best_objective：reg:gamma，best_max_depth：10，best_n_estimators：410，best_learning_rate：0.1，加权预测误差绝对值的均值：0.16742737247132877，最小误差绝对值总和为：147.16866040229797。
预测：2020-02-04，预测误差百分比总和：4.983532261813344，预测差总和：-88.33940863609314，预测差绝对值的总和：147.16866040229797，加权预测误差绝对值的均值：0.16742737247132877


In [23]:
print('每日新增预测说明：除了港澳台、湖北外的省、直辖市，下一日新增确诊人数预测，'
      '2020-02-04 表示从 2020-02-04 凌晨 到 2020-02-05 凌晨之间新增人数。'
      '数据采集自丁香医生网每天各地每天第一条确诊人数变化的数据。')
pd.set_option('max_colwidth',100)
dfs = []
for date in sorted(list(dfs_predict.keys())):
    df = dfs_predict[date][['预测', '实际']]
    df.columns = pd.MultiIndex.from_product([[date], df.columns])
    dfs.append(df)
df = pd.concat(dfs, axis=1, sort=False)
df = df * 100000
df = df.astype(np.int32)
df = df.astype(np.float32)
df = df / 100000
df = df.iloc[:, [1, 2]]
# df.loc[:('2020-02-04', 'a')] = df.iloc[1] - df.iloc[0]
df.columns = ['2020-02-03实际', '2020-02-04预测']
mask = df.iloc[:, 1] - df.iloc[:, 0]
l = []
for m in mask.values:
    if m < 0:
        l.append('降')
    elif m == 0:
        l.append('平')
    else:
        l.append('升')
df['相比前日'] = l
df

每日新增预测说明：除了港澳台、湖北外的省、直辖市，下一日新增确诊人数预测，2020-02-04 表示从 2020-02-04 凌晨 到 2020-02-05 凌晨之间新增人数。数据采集自丁香医生网每天各地每天第一条确诊人数变化的数据。


,2020-02-03实际,2020-02-04预测,相比前日
广东,114.0,77.07632,降
河南,109.0,87.58338,降
浙江,105.0,93.21434,降
江西,85.0,73.04395,降
湖南,72.0,66.58438,降
安徽,72.0,61.57118,降
重庆,37.0,41.50994,升
江苏,37.0,39.07899,升
黑龙江,37.0,31.07209,降
四川,28.0,25.18164,降
